In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.framework import ops

from load_data import *
from models.customlayers import *
from models.activations import *
from training import *

import models.ConvAE as cae
L = tf.layers

import matplotlib.pyplot as plt
% matplotlib inline

# Model

Try first:

- input full video, output full video
- lstm encoder-decoder: at each timestep, input frame, output prediction for next frame
- start with fixed size: 20 frames (later work with true recurrent net)

Below is wrong; just instantiate lstm outside of function for now


In [ ]:
input_var = tf.placeholder(dtype=tf.float32, shape=(20, 3, 60, 80), name='input')
target_var = tf.placeholder(dtype=tf.float32, shape=(20, 3, 60, 80), name='target')

def unwrapped_transition(num_units=1024, num_steps=19):
    state = tf.zeros([num_steps, num_units])
    initial_state = state
    
    with tf.variable_scope('encoder'):
        encoded = cae.encoder(input_var)
        
    with tf.variable_scope('lstm'):
        lstm = tf.nn.rnn_cell.LSTMCell(num_units=num_units, use_peepholes=True, activation=selu)
    
    current_loss = 0
    for step in range(num_steps):
        output, state = lstm(encoded[i, :], state)
        
        current_loss += tf.reduce_mean(tf.pow(output - encoded[i+1], 2))

In [3]:
def transition(encoded, num_units=1024, num_steps=20):
    input_shape = encoded.get_shape().as_list()
    state = tf.zeros([input_shape[0], num_units])
    initial_state = state
    encoded = []
    
    lstm = tf.nn.rnn_cell.LSTMCell(num_units=num_units, use_peepholes=True, activation=selu)
    
    for step in range(num_steps):
        output, state = lstm(encoded[step, :], state)
        
    final_state = state
    
    return 

In [ ]:
tf.reset_default_graph()

input_var = tf.placeholder(dtype=tf.float32, shape=(None, 3, 60, 80), name='input')
target_var = tf.placeholder(dtype=tf.float32, shape=(None, 3, 60, 80), name='target')

with tf.variable_scope('encoder'):
    encoded = cae.encoder(input_var)

with tf.variable_scope('lstm'):
    latent = encoded # just pass for now

with tf.variable_scope('decoder'):
    decoded0 = cae.decoder(encoded)
    
with tf.variable_scope('decoder', reuse=True):
    decoded1 = cae.decoder(latent)

loss = tf.reduce_mean(tf.pow(decoded1 - target_var, 2)) + tf.reduce_mean(tf.pow(decoded0 - input_var, 2))
train_step = tf.train.AdamOptimizer().minimize(loss)
saver = tf.train.Saver()
init = tf.global_variables_initializer()